# QS to grab
using XHR gotten in urls like https://www.topuniversities.com/university-rankings/world-university-rankings/2022

In [5]:
# 从network的xhr文件中发现json文件的url
urls={
    '2022': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/3740566.txt?1623183713?v=1623251204159',
    '2021': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/2057712.txt?1623183364?v=1623254149518',
    '2020': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/914824.txt?1623183839?v=1623254203592',
    '2019': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/397863.txt?1623183911?v=1623254242977',
    '2018': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/357051.txt?1623189642?v=1623254394658',
    '2017': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/357051.txt?1623189642?v=1623254422154',
    '2016': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/326584.txt?1623203022?v=1623254459114',
    '2015': 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/en/299926.txt?1623193600?v=1623254527088'
}

In [ ]:
# crawl
import requests
import json
for year, url in urls.items():
    cont = requests.get(url)
    data = cont.json()   # json数据类型为dict
    with open(r'.\QS-json\{0}.json'.format(year), 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))

# QS to analyse
process data collected from qs website

In [19]:
import requests
import json
import xlsxwriter as xlwt
from bs4 import BeautifulSoup
# 将字典数据转换为数组
def parser_page(json):
    if json:
        items = json.get('data')
        for i in range(len(items)):
            item = items[i]
            QSrank = {}
            QSrank['title'] = BeautifulSoup(item['title']).get_text()
            if "=" in item['rank_display']:
                rk_str = str(item['rank_display']).split('=')[-1]
                QSrank['rank_display'] = rk_str
            else:
                QSrank['rank_display'] = item['rank_display']
            QSrank['region'] = item['region']
            QSrank['score'] = item['score']
            yield QSrank

workbook = xlwt.Workbook(r'.\QS.xlsx')
worksheet_main = workbook.add_worksheet('main')

# 写每一年的分表
for year in urls.keys():
    with open(r'.\QS-json\{0}.json'.format(year), 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
    results = parser_page(data)

    worksheet = workbook.add_worksheet(year)

    headings = ['Title', 'Rank', 'Region', 'Score']
    col = 0
    for heading in headings:
        worksheet.write(0, col, heading)
        col = col+1

    row = 1
    for result in results:
        col = 0
        for k, v in result.items():
            worksheet.write(row, col, v)
            col = col+1
        row = row+1

# 写 main 表
with open(r'.\QS-json\{0}.json'.format(2022), 'r', encoding='utf-8') as f:
    data = json.loads(f.read())
results = parser_page(data)
headings_main = ['Title'] + list(urls.keys())
col = 0
for heading in headings_main:
    worksheet_main.write(0, col, heading)
    col += 1
row = 1
for result in results:
    worksheet_main.write(row, 0, result['title'])
    for year in urls.keys():
        worksheet_main.write(
            row, headings_main.index(year), "=VLOOKUP(A{0},'{1}'!A:B,2,0)".format(row + 1, year))
    row += 1

worksheet_main.set_column('A:A',20) 
workbook.close()

# THE data getting
like https://www.timeshighereducation.com/world-university-rankings/2021/world-ranking
THE 的网页和QS一样都把数据存在 XHR 里，所以找到对应的 json 文件链接便可获取完整数据。但要额外注意的是 THE 的网站对 headers 有检查，不加 headers 会返回 403。

In [5]:
# 从network的xhr文件中发现json文件的url
urls={
    '2021': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2021_0__fa224219a267a5b9c4287386a97c70ea.json',
    '2020': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2020_0__24cc3874b05eea134ee2716dbf93f11a.json',
    '2019': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2019_0__8923a34186e552aa8aec863e45bc02d5.json',
    '2018': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_0__e814f039fcc8ddc45dc6085e4a8a8b66.json',
    '2017': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2017_0__06ec07a66faf58bb6171791e5852fe1c.json',
    '2016': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2016_0__db6aad7f77c771c83817ced2b2ed3722.json',
    '2015': 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2015_0__0ea44da985e26d09045368c98f819b91.json'
}
latest_year = list(urls.keys())[0]

In [4]:
# crawl
import requests
import json
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
for year, url in urls.items():
    cont = requests.get(url, headers=headers)
    print(cont)
    data = cont.json()   # json数据类型为dict
    with open(r'.\THE-json\{0}.json'.format(year), 'w', encoding='utf-8') as f:
        f.write(json.dumps(data))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


# THE to analyse

In [6]:
import requests
import json
import xlsxwriter as xlwt
from bs4 import BeautifulSoup
# 将字典数据转换为数组
def parser_page(json):
    if json:
        items = json.get('data')
        for i in range(len(items)):
            item = items[i]
            res = {}
            res['title'] = BeautifulSoup(item['name']).get_text()
            if "=" in item['rank']:
                rk_str = str(item['rank']).split('=')[-1]
                res['rank_display'] = rk_str
            else:
                res['rank_display'] = item['rank']
            res['region'] = item['location']
            res['score'] = item['scores_overall']
            res['aliases'] = item['aliases']
            yield res

workbook = xlwt.Workbook(r'.\THE.xlsx')
worksheet_main = workbook.add_worksheet('main')

# 写每一年的分表
for year in urls.keys():
    with open(r'.\THE-json\{0}.json'.format(year), 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
    results = parser_page(data)

    worksheet = workbook.add_worksheet(year)

    headings = ['Title', 'Rank', 'Region', 'Score', 'Aliases']
    col = 0
    for heading in headings:
        worksheet.write(0, col, heading)
        col = col+1

    row = 1
    for result in results:
        col = 0
        for k, v in result.items():
            worksheet.write(row, col, v)
            col = col+1
        row = row+1

# 写 main 表
with open(r'.\THE-json\{0}.json'.format(latest_year), 'r', encoding='utf-8') as f:
    data = json.loads(f.read())
results = parser_page(data)
headings_main = ['Title'] + list(urls.keys())
col = 0
for heading in headings_main:
    worksheet_main.write(0, col, heading)
    col += 1
row = 1
for result in results:
    worksheet_main.write(row, 0, result['title'])
    for year in urls.keys():
        worksheet_main.write(
            row, headings_main.index(year), "=VLOOKUP(A{0},'{1}'!A:B,2,0)".format(row + 1, year))
    row += 1

worksheet_main.set_column('A:A',20) 
workbook.close()